![](imgs/miredtwitter.png)  
<center>
<h1>
Recomendación de Información Basada en<br />
Análisis de Redes Sociales<br />
y Procesamiento de Lenguaje Natural
</h1>

<h2>Trabajo Final de la Licenciatura en Ciencias de la Computación</h2>

<h3>Pablo Gabriel Celayes</h3>
<h3>Martes 30 de mayo de 2017</h3>
</center>

# Introducción
## Idea original

- Recomendador *personalizado* de artículos basado en contenido ( NLP )
- Mejorarlo con información social ( de fuentes externas o relaciones *inferidas* ) 
- Preferencias de usuarios de Cogfor

![](imgs/cogfor.png)

# Intro
## Mutación

- Set the datos propio ( usuarios, preferencias, conexiones ) 
- Predecir preferencias usando información social
- Combinar con recomendación basada en contenido


# Intro
## Menú de hoy
- Set the datos propio ( usuarios, preferencias, conexiones ) 
- Predecir preferencias usando información social
- ~~Combinar con recomendación basada en contenido~~


# Herramientas
![](imgs/tweepy.png)
![](imgs/sqlalchemy.png)
![](imgs/sklearn.png)
![](imgs/networkx.png)
![](imgs/graphtool.png)
![](imgs/pandas.png)
![](imgs/jupyter.png)


# Datos: Grafo social

- hasta 3 pasos de la relación **seguir** empezando de mí mismo
- filtramos usuarios **relevantes** ( >40 seguidos/seguidores )
- 3M usuarios
- 10M conexiones


![](imgs/miredtwitter.png)


# Datos: Contenido
- Subgrafo:
    - partimos de un conjunto pequeño de usuarios
    - agregamos 50 *seguidos* más *afines* ( según *seguidos* en común )
    - repetimos hasta que no hay nuevos
- 5180 usuarios


# Datos: Contenido

- *timelines* de 2 meses (+ *retweets* y *favs*)
- 2M tweets

# Experimento social

¿ Dado un usuario, cuanto puedo saber de sus preferencias sabiendo las de su entorno ?

- **Contenido** =  tweets en español
- **Preferencias** = retweets
- **Entorno** = seguidos + seguidos-por-seguidos
- **Usuarios** = más activos y más centrales ( ~200 )
- **Clasificador** = *Support Vector Machines*


In [4]:
from experiments.datasets import *
uid = 228252737
X_train, X_test, y_train, y_test = load_dataframe(uid)
X_train.shape

(155904, 625)

In [5]:
X_train.head()

,129102317,214242743,449712065,28631631,157227260,49846983,153932567,150067746,563863289,205088752,...,18430415,213261375,151299316,175914955,913020996,181331362,82088561,146054202,215872828,153169998
638883041042145280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
642016953385676802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
636343628365963264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
640971342804856836,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
637701998658367488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Matrices bastante *ralas*
X_train.sum().sum() / (X_train.shape[0] * X_train.shape[1])

0.0017730603448275861

In [ ]:
# Balanceando clases
w1 = sum(y_train)/len(y_train)
w0 = 1 - w1
sample_weights = np.array([w0 if x==0 else w1 for x in y_train])

# Entrenamos clasificador
clf = RandomForestClassifier()     
clf.fit(X_train, y_train, sample_weight=sample_weights)

In [12]:
# Y evaluamos...
from sklearn.metrics import classification_report
def evaluate_model(clf, X_train, X_test, y_train, y_test):
    y_true, y_pred = y_train, clf.predict(X_train)

    print("Detailed classification report:\n")
    print("Scores on training set.\n")
    print(classification_report(y_true, y_pred))

    y_true, y_pred = y_test, clf.predict(X_test)
    print("Scores on test set.\n")
    print(classification_report(y_true, y_pred))

evaluate_model(clf, X_train, X_test, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00    154277
        1.0       1.00      0.81      0.90      1627

avg / total       1.00      1.00      1.00    155904

Scores on test set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     66153
        1.0       0.96      0.76      0.85       664

avg / total       1.00      1.00      1.00     66817



# ¿Qué pasó con los demás?

- En general los mismos buenos resultados
- El punto crucial es el *recall* de la clase positiva

$F1$ promedio $0.84$

# TODO: poner cuartiles de F1
![](imgs/f1svalidsocial.png)


# Agregando PLN


## Selección de usuarios

## Pre-procesamiento

- Normalización
- Tokenizado
- Extracción de frases
- Diccionario de términos
- *Bag of terms*
- LDA


In [ ]:
## Normalización

In [ ]:
## Tokenizado

## Extracción de frases

2 pasadas del $Phraser$ de $gensim$

Se corrió este algoritmo sobre el corpus completo $T$ previamente tokenizado, ignorando
todas las palabras y frases que ocurren menos de $min\_count = 5$ veces y tomando como frases relevantes
aquellos bigramas $a b$ tales que:

$$ \frac{(cnt(a, b) - min\_count) * N}{cnt(a) * cnt(b)} > 10 $$


![](imgs/frases.png)


In [ ]:
## Diccionario de términos

## *Bag of terms*

- Texto $t$:  $\rightarrow$ multiconjunto (*bag*) de los términos en $t$.
- No importa el orden, pero sí repeticiones.
- En tweets ( $\leq 140$ caracteres ), en general son *conjuntos* ( $0$ o $1$ ocurrencia).
- Diccionario fijo $D = \{ t_1, \ldots, t_{26201} \}$ : $\rightarrow$ vector de características enteras (booleanas):

$$ v_{BOT}(tweet) = [count(t_i, tokens(tweet))]_{i=1}^{26201} $$

- $v_{BOT}$ se representa ralo (*sparse*):

![](imgs/bagofterms.png)


In [ ]:
import pickle
import pyLDAvis

n_topics = 10
viz = pickle.load(open("../experiments/_60_nlpmodels/tweets_es_%d.viz" % n_topics,'rb'))

In [3]:
pyLDAvis.display(viz)

## LDA

![](imgs/lda10.png)


# Clasificación social + PLN

![](imgs/f1lda_eval.png)


## Sobreajuste
![](imgs/f1lda_train.png)


## Discretizando...
![](imgs/f1ldabool_eval.png)


# Próximos pasos

- Reducir sobreajuste
- Características adicionales
- Considerar temporalidad
- Generalizar ( modelo que no dependa del usuario )


# ¿Preguntas?

![](imgs/twitter.png) @PCelayes

![](imgs/github.jpg) https://github.com/pablocelayes/sna_classifier

